In [10]:
import numpy as np

In [18]:
import pymysql
import pandas as pd

# Configuration de la connexion MySQL
DB_CONFIG = {
    "host": "localhost",  # Modifier selon votre configuration
    "user": "root",  # Modifier avec votre utilisateur MySQL
    "password": "password",  # Modifier avec votre mot de passe MySQL
    "database": "hl7_database"
}

# Liste des fichiers CSV générés
csv_files = {
    "MSH": "MSH_export.csv",
    "EVN": "EVN_export.csv",
    "PID": "PID_export.csv",
    "PD1": "PD1_export.csv",
    "ZPA": "ZPA_export.csv",
    "PV1": "PV1_export.csv",
    "PV2": "PV2_export.csv",
    "IN1": "IN1_export.csv",
    "ZIN": "ZIN_export.csv"
}

# Création des tables correspondantes
def create_tables(cursor):
    tables = {
        "MSH": "CREATE TABLE IF NOT EXISTS MSH (id INT AUTO_INCREMENT PRIMARY KEY, Field_1 VARCHAR(255), Field_2 VARCHAR(255))",
        "EVN": "CREATE TABLE IF NOT EXISTS EVN (id INT AUTO_INCREMENT PRIMARY KEY, Field_1 VARCHAR(255), Field_2 VARCHAR(255))",
        "PID": "CREATE TABLE IF NOT EXISTS PID (id INT AUTO_INCREMENT PRIMARY KEY, Field_1 VARCHAR(255), Field_2 VARCHAR(255), Field_3 VARCHAR(255), Field_4 VARCHAR(255))",
        "PV1": "CREATE TABLE IF NOT EXISTS PV1 (id INT AUTO_INCREMENT PRIMARY KEY, Field_1 VARCHAR(255), Field_2 VARCHAR(255))",
        "IN1": "CREATE TABLE IF NOT EXISTS IN1 (id INT AUTO_INCREMENT PRIMARY KEY, Field_1 VARCHAR(255), Field_2 VARCHAR(255))"
    }
    for table, query in tables.items():
        cursor.execute(query)

# Fonction pour importer un fichier CSV dans une table MySQL
def import_csv_to_mysql(cursor, table_name, file_path):
    df = pd.read_csv(file_path)
    columns = ", ".join(df.columns)
    values_placeholders = ", ".join(["%s"] * len(df.columns))
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_placeholders})"
    
    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))

# Connexion et importation
def main():
    connection = pymysql.connect(**DB_CONFIG)
    cursor = connection.cursor()
    
    create_tables(cursor)  # Création des tables
    
    for table, file in csv_files.items():
        import_csv_to_mysql(cursor, table)
        print(f"Données importées dans {table}")
    
    connection.commit()
    cursor.close()
    connection.close()

if __name__ == "__main__":
    main()


OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée)")

In [8]:
import hl7
import pandas as pd
import psycopg2
from psycopg2 import sql, OperationalError
import time
import logging
import os
import shutil

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Connexion à la base PostgreSQL
def connect_db(dbname):
    try:
        conn = psycopg2.connect(
            dbname=dbname,
            user='postgres',
            password='password',
            host='localhost',
            port='5432'
        )
        logging.info(f"✅ Connexion à la base {dbname} réussie")
        return conn
    except OperationalError as e:
        logging.error(f"❌ Erreur lors de la connexion à PostgreSQL ({dbname}): {e}")
        return None

# Création des tables
def create_tables(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Patients (
            id SERIAL PRIMARY KEY,
            patient_id VARCHAR(255) UNIQUE,
            name VARCHAR(255),
            birth_date DATE,
            gender VARCHAR(10),
            service VARCHAR(255),
            status VARCHAR(255)
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Operations (
            id SERIAL PRIMARY KEY,
            patient_id VARCHAR(255),
            operation_code VARCHAR(255),
            operation_date DATE,
            doctor VARCHAR(255),
            department VARCHAR(255),
            FOREIGN KEY (patient_id) REFERENCES Patients(patient_id)
        )
    ''')
    conn.commit()
    logging.info("✅ Tables créées avec succès")

# Fonction pour exporter les données en CSV
def export_to_csv(conn, dbname):
    try:
        patients_query = "SELECT * FROM Patients"
        operations_query = "SELECT * FROM Operations"
        
        patients_df = pd.read_sql(patients_query, conn)
        operations_df = pd.read_sql(operations_query, conn)
        
        patients_df.to_csv(f"{dbname}_patients_data.csv", index=False)
        operations_df.to_csv(f"{dbname}_operations_data.csv", index=False)
        
        logging.info(f"✅ Données exportées en CSV pour {dbname}")
    except Exception as e:
        logging.error(f"❌ Erreur lors de l'export CSV ({dbname}): {e}")

# Fonction pour parser et stocker les messages HL7 selon leur type
def parse_and_store_hl7_message(message, conn, dbname):
    try:
        parsed_message = hl7.parse(message)
        msh = parsed_message.segment("MSH")
        message_type = msh[9][0]
        
        if message_type.startswith("ADT"):
            dbname = "WISH"
        elif message_type.startswith("ORU"):
            dbname = "ORLine"
        
        conn = connect_db(dbname)
        create_tables(conn)
        
        pid = parsed_message.segment("PID")
        obr = parsed_message.segment("OBR") if "OBR" in parsed_message else None
        
        patient_id = pid[3][0]
        patient_name = pid[5][0]
        birth_date = pid[7][0]
        gender = pid[8][0]
        service = "Emergency"
        status = "Admitted"
        operation_code = obr[4][0] if obr else None
        operation_date = obr[7][0] if obr else None
        doctor = obr[16][0] if obr else None
        department = "Surgery" if obr else None
    
        cursor = conn.cursor()
        
        # Insertion patient (ignore si déjà présent)
        cursor.execute('''
            INSERT INTO Patients (patient_id, name, birth_date, gender, service, status)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (patient_id) DO NOTHING
        ''', (patient_id, patient_name, birth_date, gender, service, status))
        
        # Insertion opération
        if operation_code:
            cursor.execute('''
                INSERT INTO Operations (patient_id, operation_code, operation_date, doctor, department)
                VALUES (%s, %s, %s, %s, %s)
            ''', (patient_id, operation_code, operation_date, doctor, department))
        
        conn.commit()
        logging.info(f"✅ Message HL7 ({message_type}) stocké pour le patient {patient_name} dans {dbname}")
    except Exception as e:
        logging.error(f"❌ Erreur lors du parsing du message HL7 : {e}")

# Traitement des fichiers HL7 et archivage
def process_existing_hl7_files(directory_path, archive_path):
    if not os.path.exists(directory_path):
        logging.error(f"❌ Le répertoire spécifié n'existe pas: {directory_path}")
        return
    if not os.path.exists(archive_path):
        os.makedirs(archive_path)
        
    for filename in os.listdir(directory_path):
        if filename.endswith(".hl7"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                hl7_message = file.read()
                parse_and_store_hl7_message(hl7_message, None, "")
            
            shutil.move(file_path, os.path.join(archive_path, filename))
            logging.info(f"✅ Fichier HL7 archivé : {filename}")

if __name__ == "__main__":
    hl7_directory = "C:/Users/sbenayed/Desktop/PFE/Data"
    archive_directory = "./hl7_archive"
    process_existing_hl7_files(hl7_directory, archive_directory)


2025-02-14 11:52:53.274 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 11:52:53.976 
  command:

    streamlit run c:\Users\sbenayed\Desktop\PFE\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-14 11:52:53.978 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 11:52:53.981 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 11:52:53.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 11:52:53.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 11:52:53.984 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 11:52:53.985 Thread 'MainT